<h1 style="color:lightblue;"><b><u>Import Library</u></b></h1>

<h3 style="color:lightblue;">Kita akan mengimpor semua "alat" yang kita butuhkan dari sklearn.</h3>

In [16]:
import pandas as pd
import joblib # Untuk menyimpan model
import os

# --- Preprocessing ---
from sklearn.model_selection import train_test_split # Untuk membagi data
from sklearn.preprocessing import OneHotEncoder # Untuk ubah Kategori -> Angka
from sklearn.compose import ColumnTransformer # Untuk 'menggabungkan' preprocessing

# --- Model ---
from sklearn.linear_model import LinearRegression # Model Regresi
from sklearn.ensemble import RandomForestClassifier # Model Klasifikasi

# --- Evaluasi ---
from sklearn.metrics import mean_squared_error, r2_score # Metrik Regresi
from sklearn.metrics import accuracy_score, classification_report # Metrik Klasifikasi

print("Libraries berhasil di-import.")

Libraries berhasil di-import.


<h1 style="color:lightblue;"><b><u>Muat Data</u></b></h1>

<h3 style="color:lightblue;">Kita gunakan data dari Langkah Feature Engineering (SuperStore_Processed.csv).</h3>

In [17]:
processed_data_path = "../data/SuperStore_Processed.csv"
df = pd.read_csv(processed_data_path)

# Cek data
print(f"Data dimuat, jumlah baris: {len(df)}")
df.head()

Data dimuat, jumlah baris: 9993


,Order_ID,Customer_ID,Postal_Code,Product_ID,Sales,Quantity,Discount,Profit,Category,Sub-Category,...,City,State,Region,Order_Year,Order_Month,Order_Month_Name,Order_Day_Name,Shipping_Duration_Days,Profit_Margin_%,Profit_Status
0,CA-2019-152156,CG-12520,42420,FUR-BO-10001798,261.9600,2,0.00,41.9136,Furniture,Bookcases,...,Henderson,Kentucky,South,2019,11,November,Friday,3,16.00,Profitable
1,CA-2019-152156,CG-12520,42420,FUR-CH-10000454,731.9400,3,0.00,219.5820,Furniture,Chairs,...,Henderson,Kentucky,South,2019,11,November,Friday,3,30.00,Profitable
2,CA-2019-138688,DV-13045,90036,OFF-LA-10000240,14.6200,2,0.00,6.8714,Office Supplies,Labels,...,Los Angeles,California,West,2019,6,June,Wednesday,4,47.00,Profitable
3,US-2018-108966,SO-20335,33311,FUR-TA-10000577,957.5775,5,0.45,-383.0310,Furniture,Tables,...,Fort Lauderdale,Florida,South,2018,10,October,Thursday,7,-40.00,Not Profitable
4,US-2018-108966,SO-20335,33311,OFF-ST-10000760,22.3680,2,0.20,2.5164,Office Supplies,Storage,...,Fort Lauderdale,Florida,South,2018,10,October,Thursday,7,11.25,Profitable


<h1 style="color:lightblue;"><b><u>Persiapan Data untuk Klasifikasi</u></b></h1>

<h3 style="color:lightblue;">Model klasifikasi Anda (Is_Profitable) butuh target Ya/Tidak. Kita buat kolom biner (1 = Ya, 0 = Tidak) dari kolom Profit.</h3>

In [18]:
# Membuat target biner: 1 jika Profit > 0, 0 jika tidak (rugi atau impas)
df['Is_Profitable'] = (df['Profit'] > 0).astype(int)

print("Kolom 'Is_Profitable' berhasil dibuat.")
df[['Profit', 'Is_Profitable']].head(10)

Kolom 'Is_Profitable' berhasil dibuat.


,Profit,Is_Profitable
0,41.9136,1
1,219.5820,1
2,6.8714,1
3,-383.0310,0
4,2.5164,1
5,14.1694,1
6,1.9656,1
7,90.7152,1
8,5.7825,1
9,34.4700,1


<h1 style="color:lightblue;"><b><u>Bagian 1 : Model Regresi (Memprediksi Profit)</u></b></h1>

<h3 style="color:lightblue;">Kita akan memprediksi angka Profit berdasarkan Category, Region, Quantity, dan Discount.</h3>

<h2 style="color:lightblue;"><b><u>Persiapan Data (Regresi)</u></b></h2>

<h3 style="color:lightblue;">Pilih Fitur (X) dan Target (y) sesuai permintaan Anda.</h3>

In [19]:
print("--- Memulai Model 1: REGRESI (Prediksi Profit) ---")
# Fitur (X)
features_reg = ['Category', 'Region', 'Quantity', 'Discount']
# Target (y)
target_reg = 'Profit'

X_reg = df[features_reg]
y_reg = df[target_reg]

print("Fitur (X) Regresi:")
print(X_reg.head())
print("\nTarget (y) Regresi:")
print(y_reg.head())

--- Memulai Model 1: REGRESI (Prediksi Profit) ---
Fitur (X) Regresi:
          Category Region  Quantity  Discount
0        Furniture  South         2      0.00
1        Furniture  South         3      0.00
2  Office Supplies   West         2      0.00
3        Furniture  South         5      0.45
4  Office Supplies  South         2      0.20

Target (y) Regresi:
0     41.9136
1    219.5820
2      6.8714
3   -383.0310
4      2.5164
Name: Profit, dtype: float64


<h2 style="color:lightblue;"><b><u>Preprocessing (Regresi)</u></b></h2>

<h3 style="color:lightblue;">Komputer tidak mengerti teks seperti "Furniture" atau "West". Kita harus mengubahnya menjadi angka menggunakan One-Hot Encoding.</h3>

In [20]:
# Tentukan mana kolom kategori dan mana kolom numerik
categorical_cols = ['Category', 'Region']
numerical_cols = ['Quantity', 'Discount']

# Buat 'transformer'
# 'OneHotEncoder' untuk kolom kategori
# 'passthrough' untuk kolom numerik (biarkan apa adanya)
preprocessor_reg = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', 'passthrough', numerical_cols)
    ],
    remainder='drop' # Abaikan kolom lain
)

# Terapkan preprocessor ke data X kita
X_reg_processed = preprocessor_reg.fit_transform(X_reg)

print(f"Data X berhasil diproses. Shape baru: {X_reg_processed.shape}")

Data X berhasil diproses. Shape baru: (9993, 9)


<h2 style="color:lightblue;"><b><u>Bagi Data (Train & Test)</u></b></h2>

<h3 style="color:lightblue;">Kita akan melatih model pada 80% data (train set) dan mengujinya pada 20% data (test set) yang belum pernah dilihatnya.</h3>

In [21]:
# Bagi data: 80% untuk latih, 20% untuk tes
X_train, X_test, y_train, y_test = train_test_split(
    X_reg_processed, y_reg, 
    test_size=0.2, 
    random_state=42 # 'random_state' agar hasilnya konsisten
)

print(f"Data Latih: {X_train.shape[0]} baris")
print(f"Data Tes: {X_test.shape[0]} baris")

Data Latih: 7994 baris
Data Tes: 1999 baris


<h2 style="color:lightblue;"><b><u>Latih Model (Regresi)</u></b></h2>

<h3 style="color:lightblue;">Disini saatnya kita melatih LinearRegression.</h3>

In [22]:
# 1. Inisiasi model
model_reg = LinearRegression()

# 2. Latih model dengan data latih (X_train, y_train)
print("Melatih model Linear Regression...")
model_reg.fit(X_train, y_train)

print("-> Model Regresi berhasil dilatih!")

Melatih model Linear Regression...
-> Model Regresi berhasil dilatih!


<h2 style="color:lightblue;"><b><u>Evaluasi Model (Regresi)</u></b></h2>

<h3 style="color:lightblue;">Seberapa bagus model kita? Kita akan gunakan dua metrik:<p>1. RMSE (Root Mean Squared Error): Rata-rata error prediksi (semakin kecil, semakin baik).</p>2. R-squared ($R^2$): Seberapa banyak variasi Profit yang bisa dijelaskan model (0-1, semakin dekat ke 1, semakin baik).</h3>

In [23]:
# Buat prediksi di data tes
y_pred_reg = model_reg.predict(X_test)

# 1. Hitung Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred_reg)
# 2. Hitung RMSE dengan cara mengakarkan MSE (pangkat 0.5)
rmse = mse ** 0.5

# Hitung R-squared
r2 = r2_score(y_test, y_pred_reg)

print("--- Hasil Evaluasi Model Regresi ---")
print(f"RMSE (Rata-rata Error): ${rmse:,.2f}")
print(f"R-squared ($R^2$): {r2:.2f}")

print(f"\nArtinya: Model ini bisa memprediksi Profit dengan rata-rata error ${rmse:,.2f}.")
print(f"Dan model ini bisa menjelaskan {r2*100:.1f}% variasi dari data Profit.")

--- Hasil Evaluasi Model Regresi ---
RMSE (Rata-rata Error): $278.14
R-squared ($R^2$): 0.05

Artinya: Model ini bisa memprediksi Profit dengan rata-rata error $278.14.
Dan model ini bisa menjelaskan 4.9% variasi dari data Profit.


<h1 style="color:lightblue;"><b><u>Bagian 2: Model Klasifikasi (Memprediksi Untung/Rugi)</u></b></h1>

<h3 style="color:lightblue;">Kita akan memprediksi kategori Is_Profitable (1 atau 0) berdasarkan Category, Discount, Region.</h3>

<h2 style="color:lightblue;"><b><u>Persiapan Data (Klasifikasi)</u></b></h2>

<h3 style="color:lightblue;">Pilih Fitur (X) dan Target (y) baru.</h3>

In [24]:
print("\n--- Memulai Model 2: KLASIFIKASI (Prediksi Untung/Rugi) ---")
# Fitur (X)
features_cls = ['Category', 'Discount', 'Region']
# Target (y)
target_cls = 'Is_Profitable' # Kolom 1/0 yg kita buat di Sel 3

X_cls = df[features_cls]
y_cls = df[target_cls]

print("Fitur (X) Klasifikasi:")
print(X_cls.head())
print("\nTarget (y) Klasifikasi:")
print(y_cls.head())


--- Memulai Model 2: KLASIFIKASI (Prediksi Untung/Rugi) ---
Fitur (X) Klasifikasi:
          Category  Discount Region
0        Furniture      0.00  South
1        Furniture      0.00  South
2  Office Supplies      0.00   West
3        Furniture      0.45  South
4  Office Supplies      0.20  South

Target (y) Klasifikasi:
0    1
1    1
2    1
3    0
4    1
Name: Is_Profitable, dtype: int64


<h2 style="color:lightblue;"><b><u>Preprocessing (Klasifikasi)</u></b></h2>

<h3 style="color:lightblue;">Sama seperti regresi, kita perlu melakukan One-Hot Encoding untuk Category dan Region.</h3>

In [25]:
# Tentukan kolom
categorical_cols_cls = ['Category', 'Region']
numerical_cols_cls = ['Discount']

# Buat preprocessor
preprocessor_cls = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols_cls),
        ('num', 'passthrough', numerical_cols_cls)
    ],
    remainder='drop'
)

# Terapkan ke data X
X_cls_processed = preprocessor_cls.fit_transform(X_cls)

print(f"Data X Klasifikasi berhasil diproses. Shape baru: {X_cls_processed.shape}")

Data X Klasifikasi berhasil diproses. Shape baru: (9993, 8)


<h2 style="color:lightblue;"><b><u>Bagi Data</u></b></h2>

<h3 style="color:lightblue;">Kita akan membagi data dengan 80% akan kita latih dan 20% untuk testing</h3>

In [26]:
# Bagi data: 80% latih, 20% tes
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_cls_processed, y_cls, 
    test_size=0.2, 
    random_state=42
)

print(f"Data Latih: {X_train_cls.shape[0]} baris")
print(f"Data Tes: {X_test_cls.shape[0]} baris")

Data Latih: 7994 baris
Data Tes: 1999 baris


<h2 style="color:lightblue;"><b><u>Latih Model (Klasifikasi)</u></b></h2>

<h3 style="color:lightblue;">Kita gunakan RandomForestClassifier, model yang powerful dan bagus untuk pemula.</h3>

In [27]:
# 1. Inisiasi model
model_cls = RandomForestClassifier(n_estimators=100, random_state=42)

# 2. Latih model
print("Melatih model RandomForest Classifier...")
model_cls.fit(X_train_cls, y_train_cls)

print("-> Model Klasifikasi berhasil dilatih!")

Melatih model RandomForest Classifier...
-> Model Klasifikasi berhasil dilatih!


<h2 style="color:lightblue;"><b><u>Evaluasi Model (Klasifikasi)</u></b></h2>

<h3 style="color:lightblue;">Seberapa akurat model kita dalam menebak "Untung" vs "Rugi"?
<p>1. Accuracy: Berapa persen tebakan yang benar? (semakin tinggi, semakin baik).</p>
2. Classification Report: Laporan detail (Precision, Recall) untuk setiap kelas (0 dan 1).</h3>

In [28]:
# Buat prediksi di data tes
y_pred_cls = model_cls.predict(X_test_cls)

# Hitung metrik
accuracy = accuracy_score(y_test_cls, y_pred_cls)
report = classification_report(y_test_cls, y_pred_cls, target_names=['Rugi (0)', 'Untung (1)'])

print("--- Hasil Evaluasi Model Klasifikasi ---")
print(f"Akurasi: {accuracy * 100:.2f}%")
print(f"\nArtinya: Model ini berhasil menebak {accuracy * 100:.2f}% data tes dengan benar.")

print("\nLaporan Detail (Classification Report):")
print(report)

--- Hasil Evaluasi Model Klasifikasi ---
Akurasi: 94.80%

Artinya: Model ini berhasil menebak 94.80% data tes dengan benar.

Laporan Detail (Classification Report):
              precision    recall  f1-score   support

    Rugi (0)       0.99      0.73      0.84       378
  Untung (1)       0.94      1.00      0.97      1621

    accuracy                           0.95      1999
   macro avg       0.97      0.87      0.91      1999
weighted avg       0.95      0.95      0.94      1999



<h2 style="color:lightblue;"><b><u>Simpan Model</u></b></h2>

<h3 style="color:lightblue;">kita simpan model yang sudah dilatih ke folder models/.</h3>

In [29]:
# Tentukan path folder
model_path = "../models/"
os.makedirs(model_path, exist_ok=True)

# Nama file model
file_model_reg = os.path.join(model_path, "1_regression_profit_predictor.joblib")
file_model_cls = os.path.join(model_path, "2_classification_profit_predictor.joblib")

# Simpan model
joblib.dump(model_reg, file_model_reg)
joblib.dump(preprocessor_reg, os.path.join(model_path, "1_preprocessor_reg.joblib")) # Simpan juga preprocessor-nya!

joblib.dump(model_cls, file_model_cls)
joblib.dump(preprocessor_cls, os.path.join(model_path, "2_preprocessor_cls.joblib"))

print(f"--- Model Selesai Disimpan ---")
print(f"Model Regresi disimpan di: {file_model_reg}")
print(f"Model Klasifikasi disimpan di: {file_model_cls}")
print("Preprocessor juga ikut disimpan.")

--- Model Selesai Disimpan ---
Model Regresi disimpan di: ../models/1_regression_profit_predictor.joblib
Model Klasifikasi disimpan di: ../models/2_classification_profit_predictor.joblib
Preprocessor juga ikut disimpan.
